In [1]:
import pm4py
import io
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns

from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.algo.discovery.alpha import algorithm as alpha_miner
from pm4py.visualization.petri_net import visualizer as pn_visualizer
from pm4py.visualization.bpmn import visualizer as bpmn_vis
from pm4py.statistics.traces.generic.log import case_statistics
from IPython.display import Markdown, display

In [2]:
log_domestic = pm4py.read_xes('Data/DomesticDeclarations.xes')
log_international = pm4py.read_xes('Data/InternationalDeclarations.xes')

parsing log, completed traces ::   0%|          | 0/10500 [00:00<?, ?it/s]

parsing log, completed traces ::   0%|          | 0/6449 [00:00<?, ?it/s]

In [3]:
# Funktion zur Berechnung der Aktivitätsdauer in Stunden
def calculate_activity_durations_in_hours(log):
    df = pm4py.convert_to_dataframe(log)
    df['duration'] = df.groupby('case:concept:name')['time:timestamp'].diff().dt.total_seconds()
    df['duration'] = df['duration'] / 3600  # Umrechnung in Stunden
    df['duration'] = df['duration'].round(1)  # Rundung auf eine Nachkommastelle
    return df

# Aktivitätsdauer für jeden Log berechnen
df_domestic = calculate_activity_durations_in_hours(log_domestic)
df_international = calculate_activity_durations_in_hours(log_international)

# Funktion zur Berechnung von Statistiken pro Aktivität
def activity_duration_statistics(df):
    stats = df.groupby('concept:name')['duration'].agg(['mean', 'median', 'min', 'max', 'std']).reset_index()
    frequency = df['concept:name'].value_counts().reset_index()
    frequency.columns = ['concept:name', 'frequency']
    total_frequency = frequency['frequency'].sum()
    frequency['frequency_percentage'] = (frequency['frequency'] / total_frequency) * 100
    frequency['frequency_percentage'] = frequency['frequency_percentage'].round(1)  # Rundung auf eine Nachkommastelle
    frequency['frequency_percentage'] = frequency['frequency_percentage'].astype(str) + '%'  # Hinzufügen des % Zeichens
    stats = stats.merge(frequency, on='concept:name', how='left')
    return stats

# Statistiken für inländische und internationale Aktivitäten berechnen
domestic_stats = activity_duration_statistics(df_domestic)
international_stats = activity_duration_statistics(df_international)

# Sortierung nach 'median' in absteigender Reihenfolge
domestic_stats_sorted = domestic_stats.sort_values(by='median', ascending=False)
international_stats_sorted = international_stats.sort_values(by='median', ascending=False)

# Berechnung des 90. Perzentils der Median-Dauer für inländische und internationale Aktivitäten
domestic_80th_percentile = domestic_stats_sorted['median'].quantile(0.80)
international_80th_percentile = international_stats_sorted['median'].quantile(0.80)

# Überschriften in Deutsch ändern und Formatierung anpassen
domestic_stats_sorted.rename(columns={
    'concept:name': 'Aktivität',
    'mean': 'Durchschnitt',
    'median': 'Median',
    'min': 'Minimum',
    'max': 'Maximum',
    'std': 'Standardabweichung',
    'frequency': 'Absolute Häufigkeit',
    'frequency_percentage': 'Relative Häufigkeit'
}, inplace=True)

international_stats_sorted.rename(columns={
    'concept:name': 'Aktivität',
    'mean': 'Durchschnitt',
    'median': 'Median',
    'min': 'Minimum',
    'max': 'Maximum',
    'std': 'Standardabweichung',
    'frequency': 'Absolute Häufigkeit',
    'frequency_percentage': 'Relative Häufigkeit'
}, inplace=True)

# Zahlenformate anpassen
domestic_stats_sorted['Durchschnitt'] = domestic_stats_sorted['Durchschnitt'].map('{:.1f}'.format)
domestic_stats_sorted['Standardabweichung'] = domestic_stats_sorted['Standardabweichung'].map('{:.1f}'.format)

international_stats_sorted['Durchschnitt'] = international_stats_sorted['Durchschnitt'].map('{:.1f}'.format)
international_stats_sorted['Standardabweichung'] = international_stats_sorted['Standardabweichung'].map('{:.1f}'.format)

frequency_percentage_threshold = 5

# Filter auf die sortierten Daten anwenden (Median > 80. Perzentil)
filtered_domestic_stats = domestic_stats_sorted[
    (domestic_stats_sorted['Relative Häufigkeit'].str.rstrip('%').astype(float) > frequency_percentage_threshold) &
    (domestic_stats_sorted['Median'] > domestic_80th_percentile)
]

filtered_international_stats = international_stats_sorted[
    (international_stats_sorted['Relative Häufigkeit'].str.rstrip('%').astype(float) > frequency_percentage_threshold) &
    (international_stats_sorted['Median'] > international_80th_percentile)
]

# Ergebnisse als Tabellen anzeigen
print(f"Aktivitäten im Prozess der inländischen Reiseanträge mit mindestens 5% relativer Häufigkeit und einer vergleichsweise langen Median-Dauer:")
display(filtered_domestic_stats)

print(f"Aktivitäten im Prozess der internationalen Reiseanträge mit mindestens 5% relativer Häufigkeit und einer vergleichsweise langen Median-Dauer:")
display(filtered_international_stats)

Aktivitäten im Prozess der inländischen Reiseanträge mit mindestens 5% relativer Häufigkeit und einer vergleichsweise langen Median-Dauer:


,Aktivität,Durchschnitt,Median,Minimum,Maximum,Standardabweichung,Absolute Häufigkeit,Relative Häufigkeit
15,Payment Handled,87.1,77.8,2.3,6821.6,93.1,10044,17.8%
16,Request Payment,75.9,27.4,0.0,5631.3,172.4,10040,17.8%


Aktivitäten im Prozess der internationalen Reiseanträge mit mindestens 5% relativer Häufigkeit und einer vergleichsweise langen Median-Dauer:


,Aktivität,Durchschnitt,Median,Minimum,Maximum,Standardabweichung,Absolute Häufigkeit,Relative Häufigkeit
33,Start trip,1159.3,678.5,0.1,10905.7,1684.4,6449,8.9%
30,Permit SUBMITTED by EMPLOYEE,485.7,110.1,0.0,8795.3,1056.6,6255,8.7%
14,Declaration SUBMITTED by EMPLOYEE,282.4,109.2,0.0,7968.3,598.2,8099,11.2%


[Kehre zurück zum Hauptnotebook](../0_Hauptnotebook.ipynb)